In [12]:
import pandas as pd
import re

In [2]:
df = pd.read_csv("dune_data.csv")

In [3]:
df.head()

,active_weeks,target_variable,total_volume,trader_activity_status,trader_volume_status,trader_weekly_frequency_status,tx_count_365d,wallet
0,1,🔴 Bad Trader,38.726119,🐤 Regular User,🦐 Low Value Trader,🐣 Unserious,2,0xa3c6acd45396110466fd8972875b3e6d33a0b0b8
1,1,🔴 Bad Trader,135.943754,🐣 Occasional User,🐟 Middle Value Trader,🐣 Unserious,1,0xbc0452c93e33b886d0e6750a9fc261e1d3c877ed
2,1,🔴 Bad Trader,0.457897,🐣 Occasional User,🦐 Low Value Trader,🐣 Unserious,1,0x477b0f0e29494a1e108a302ffecff3d76663386c
3,5,🔴 Bad Trader,37.614307,🐤 Regular User,🦐 Low Value Trader,🐦 OG,8,0x02170d140c843646d4802f6e65f9236baccc2f70
4,1,🔴 Bad Trader,2.389608,🐣 Occasional User,🦐 Low Value Trader,🐣 Unserious,1,0xcb69aceac5956af777d4d77c8d01c1fdea5828bd


In [10]:
df.target_variable.value_counts()

target_variable
🔴 Bad Trader     1000
🟢 Good Trader    1000
Name: count, dtype: int64

In [18]:
# Load your CSV
df = pd.read_csv("dune_data.csv")

# Define a function to remove emojis and strip whitespace
def clean_text(text):
    if isinstance(text, str):
        emoji_pattern = re.compile(
            "["
            "\U0001F600-\U0001F64F"  # emoticons
            "\U0001F300-\U0001F5FF"  # symbols & pictographs
            "\U0001F680-\U0001F6FF"  # transport & map symbols
            "\U0001F1E0-\U0001F1FF"  # flags
            "\U00002500-\U00002BEF"  # Chinese symbols
            "\U00002702-\U000027B0"
            "\U000024C2-\U0001F251"
            "\U0001f926-\U0001f937"
            "\U00010000-\U0010ffff"
            "\u2640-\u2642"
            "\u2600-\u2B55"
            "\u200d"
            "\u23cf"
            "\u23e9"
            "\u231a"
            "\ufe0f"  # dingbats
            "\u3030"
            "]+", 
            flags=re.UNICODE
        )
        # Remove emojis and trim spaces
        text = emoji_pattern.sub(r'', text)
        text = text.strip()
        return text
    else:
        return text

# Apply cleaning to all string columns
df = df.applymap(clean_text)

# Save the cleaned version
df.to_csv("cleaned_no_emojis.csv", index=False)

print("Emojis removed, whitespace stripped, and file saved as cleaned_no_emojis.csv")


Emojis removed, whitespace stripped, and file saved as cleaned_no_emojis.csv


/tmp/ipykernel_12040/1488100796.py:37: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(clean_text)


In [32]:
cleaned_df = pd.read_csv("cleaned_no_emojis.csv")

cleaned_df.head()
len(cleaned_df)

2000

In [33]:
# Load your data
df = pd.read_csv("cleaned_no_emojis.csv")

# Separate classes
good_df = df[df['target_variable'] == 'Good Trader']
bad_df = df[df['target_variable'] == 'Bad Trader']

# Desired total rows
total_rows = 1684
desired_good_ratio = 0.6

# Compute how many we *want*
desired_good = int(total_rows * desired_good_ratio)
desired_bad = total_rows - desired_good

# Adjust if not enough data
desired_good = min(desired_good, len(good_df))
desired_bad = min(desired_bad, len(bad_df))

# Rescale total if needed
actual_total = desired_good + desired_bad
print(f"Sampling {desired_good} Good Traders and {desired_bad} Bad Traders (total {actual_total})")

# Sample
good_sample = good_df.sample(n=desired_good, random_state=42)
bad_sample = bad_df.sample(n=desired_bad, random_state=42)

# Combine and shuffle
train_df = pd.concat([good_sample, bad_sample]).sample(frac=1, random_state=42).reset_index(drop=True)

# Check ratio
print(train_df['target_variable'].value_counts(normalize=True))

# Save
train_df.to_csv("train_60plus_good_trader.csv", index=False)


Sampling 1000 Good Traders and 674 Bad Traders (total 1674)
target_variable
Good Trader    0.597372
Bad Trader     0.402628
Name: proportion, dtype: float64


In [34]:
train_df.target_variable.value_counts()

target_variable
Good Trader    1000
Bad Trader      674
Name: count, dtype: int64